In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

print(torch.__version__)
print(torch.cuda.is_available())
class DepthwiseSeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DepthwiseSeparableConv2d, self).__init__()
        self.depthwise_conv = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=in_channels)
        self.pointwise_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        
    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        return x

class MobileNetEncoder(nn.Module):
    def __init__(self, in_channels=3):
        super(MobileNetEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            # nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            DepthwiseSeparableConv2d(8, 16),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=1, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            DepthwiseSeparableConv2d(16, 16),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 32, kernel_size=1, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            DepthwiseSeparableConv2d(32, 32),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=1, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.conv5 = nn.Sequential(
            DepthwiseSeparableConv2d(64, 64),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=1, stride=2, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.conv6 = nn.Sequential(
            DepthwiseSeparableConv2d(128, 128),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=1, stride=2, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x6 = self.conv6(x5)
        return x1, x2, x3, x4, x5, x6

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super(UNet, self).__init__()
        
        # Encoder
        self.encoder = MobileNetEncoder(in_channels=n_channels)
        
        # Decoder

        self.upconv5 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.iconv5 = nn.Conv2d(256, 128, kernel_size=1)
        self.decoder_block5_1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(256, 128, kernel_size=1, bias=False),
            nn.BatchNorm2d(128),
            # nn.ReLU(inplace=True),
        )
        self.decoder_block5_2 = nn.Sequential(
            nn.Conv2d(128*2, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            # nn.ReLU(inplace=True),
        )

        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.iconv4 = nn.Conv2d(128, 64, kernel_size=1)
        self.decoder_block4_1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(128, 64, kernel_size=1, bias=False),
            nn.BatchNorm2d(64)
            # nn.ReLU(inplace=True),
        )
        self.decoder_block4_2 = nn.Sequential(
            nn.Conv2d(64*2, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            # nn.ReLU(inplace=True),
        )


        self.upconv3 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.iconv3 = nn.Conv2d(64, 32, kernel_size=1)
        self.decoder_block3_1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(64, 32, kernel_size=1, bias=False),
            nn.BatchNorm2d(32)
            # nn.ReLU(inplace=True),
        )
        self.decoder_block3_2 = nn.Sequential(
            nn.Conv2d(32*2, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
        )
        
        self.upconv2 = nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2)
        self.iconv2 = nn.Conv2d(32, 16, kernel_size=1)
        self.decoder_block2_1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(32, 16, kernel_size=1, bias=False),
            nn.BatchNorm2d(16)
            # nn.ReLU(inplace=True),
        )
        self.decoder_block2_2 = nn.Sequential(
            nn.Conv2d(16*2, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            # nn.ReLU(inplace=True),
        )
        
        self.upconv1 = nn.ConvTranspose2d(16, 8, kernel_size=2, stride=2)
        # print(self.upconv1.shape)
        self.iconv1 = nn.Conv2d(16, 8, kernel_size=1)
        self.decoder_block1_1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(16, 8, kernel_size=1, bias=False),
            nn.BatchNorm2d(8)
            # nn.ReLU(inplace=True),
        )
        self.decoder_block1_2 = nn.Sequential(
            nn.Conv2d(8*2, 8, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(8),
            # nn.ReLU(inplace=True),
        )
        self.C2_layer = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=1), 
            nn.BatchNorm2d(256))
        self.C3_layer = nn.Conv2d(8, 8, kernel_size=1)
        self.C3_layer = nn.Sequential(
            nn.Conv2d(8, 8, kernel_size=1),
            nn.BatchNorm2d(8))
        self.output_layer = nn.Conv2d(8, n_classes, kernel_size=1)
            
    def forward(self, x):
        # print("Input shape: ", x.shape)
        x1, x2, x3, x4, x5, x6 = self.encoder(x)
        # print("Encoder output shapes: ", x1.shape, x2.shape, x3.shape, x4.shape, x5.shape, x6.shape)
        y6 = self.C2_layer(x6)
        # print("C2 layer shape: ", y6.shape)
        # UpSample
        y5 = self.decoder_block5_1(y6)
        # Concatenation with skip connection
        y5 = torch.cat([x5, y5], dim=1)
        # Conv2
        y5 = self.decoder_block5_2(y5)
        # print("1 - Decoder layer shape: ", y5.shape)
        # UpSample
        y4 = self.decoder_block4_1(y5)
        # Concatenation with skip connection
        y4 = torch.cat([x4, y4], dim=1)
        # Conv2
        y4 = self.decoder_block4_2(y4)
        # print("2 - Decoder layer shape: ", y4.shape)
        # UpSample
        y3 = self.decoder_block3_1(y4)
        # Concatenation with skip connection
        y3 = torch.cat([x3, y3], dim=1)
        # Conv2
        y3 = self.decoder_block3_2(y3)
        # print("3 - Decoder layer shape: ", y3.shape)
        # UpSample
        y2 = self.decoder_block2_1(y3)
        # Concatenation with skip connection
        y2 = torch.cat([x2, y2], dim=1)
        # Conv2
        y2 = self.decoder_block2_2(y2)
        # print("4 - Decoder layer shape: ", y2.shape)

        # UpSample
        y1 = self.decoder_block1_1(y2)
        # Concatenation with skip connection
        y1 = torch.cat([x1, y1], dim=1)
        # Conv2
        y1 = self.decoder_block1_2(y1)
        # print("5 - Decoder layer shape: ", y1.shape)

        out = self.output_layer(y1)
        # print("6 - Output layer shape: ", out.shape)
        return out


1.12.0+cu116
True


In [2]:
import argparse
import json
import os
import shutil
import time
import numpy as np
import random
import cv2
import os
import json
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchmetrics import F1Score,JaccardIndex


# Set seed for randomize functions (Ez reproduction of results)
random.seed(100)

# Import TuSimple loader
import sys
sys.path.insert(0,'../resources/')
from tusimple import TuSimple
import utils
# ROOT DIRECTORIES
root_dir = os.path.dirname(os.getcwd())
annotated_dir = os.path.join(root_dir,'datasets/tusimple/train_set/annotations')
clips_dir = os.path.join(root_dir,'datasets/tusimple/train_set/')
annotated = os.listdir(annotated_dir)

# Get path directories for clips and annotations for the TUSimple dataset + ground truth dictionary
annotations = list()
for gt_file in annotated:
    path = os.path.join(annotated_dir,gt_file)
    json_gt = [json.loads(line) for line in open(path)]
    annotations.append(json_gt)
    
annotations = [a for f in annotations for a in f]

dataset = TuSimple(train_annotations = annotations, train_img_dir = clips_dir, resize_to = (640,640), subset_size = 0.15, val_size= 0.01)
# Create train and validation splits / Always use del dataset to free memory after this
train_set, validation_set = dataset.train_val_split()
del dataset

In [3]:
# Custom training function for the CNN pipeline with schedule and SGD optimizer
def train(model, train_loader, val_loader = None, num_epochs=10, lr=0.01, momentum=0.9, weight_decay=1e-4, lr_scheduler=True):
    # Set up loss function and optimizer
    criterion =  nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    # set threshold to identify lane pixels to calculate eval metrics
    LANE_THRESHOLD = 0.5
    sigm = nn.Sigmoid()
    # Set up learning rate scheduler
    if lr_scheduler:
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    f1_score = F1Score(task="binary").to(device)
    iou_score = JaccardIndex(task= 'binary').to(device)
    # Train the model
    for epoch in range(num_epochs):
        # Train for one epoch
        # model.train()
        train_loss = 0
        train_iou = 0
        train_f1 = 0
        
        val_iou = 0
        val_f1 = 0
        
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            # print(targets.shape)
            # print(outputs)
            # print(outputs.shape)
            # outputs = torch.argmax(outputs, dim=1).unsqueeze(1).repeat(1,3,1,1).float()
            # outputs.requires_grad = True
            # print(outputs)
            eval_out = sigm(outputs.to(device).detach())
            eval_out = torch.where(eval_out > LANE_THRESHOLD, torch.ones_like(eval_out), torch.zeros_like(eval_out))
            # print(outputs.shape, targets.shape)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item() * inputs.size(0)
            train_iou += iou_score(eval_out, targets)
            train_f1 += f1_score(eval_out, targets)
            
        if val_loader:
            for batch_idx, (inputs, targets) in enumerate(train_loader): 
                model.eval()
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                eval_out = sigm(outputs.to(device).detach())
                eval_out = torch.where(eval_out > LANE_THRESHOLD, torch.ones_like(eval_out), torch.zeros_like(eval_out))
                # outputs = torch.argmax(outputs, dim=1).unsqueeze(1).repeat(1,3,1,1).float()
                val_iou += iou_score(eval_out, targets)
                val_f1 += f1_score(eval_out, targets)
        
            val_iou /= len(val_loader)
            val_f1 /= len(val_loader)
            
        train_loss /= len(train_loader)
        train_iou /= len(train_loader)
        train_f1 /= len(train_loader)
        
        
     # Print progress
        if lr_scheduler:
            print('Epoch: {} - Train Loss: {:.4f} - Learning Rate: {:.6f} - Train_IoU: {:.3f} - Train_F1: {:.3f}'.format(epoch+1, train_loss,scheduler.get_last_lr()[0], train_iou, train_f1))
            scheduler.step()
            if val_loader:
                print('Val_F1: {:.3f}  - Val_IoU: {:.3f} '.format(val_f1,val_iou))
        else:
            print('Epoch: {} - Train Loss: {:.4f}'.format(epoch+1, train_loss))

In [4]:
train_loader = DataLoader(train_set, batch_size=8, shuffle= True, drop_last= True)
validation_loader = DataLoader(validation_set,batch_size=8, shuffle= True, drop_last= True)  
model = UNet()
train(model, train_loader, val_loader= validation_loader, num_epochs= 5)

Epoch: 1 - Train Loss: 2.3006 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.001
Epoch: 2 - Train Loss: 0.3670 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000
Epoch: 3 - Train Loss: 0.1639 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000
Epoch: 4 - Train Loss: 0.1064 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000
Epoch: 5 - Train Loss: 0.0811 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000


In [8]:
train(model, train_loader, val_loader= validation_loader, num_epochs= 5)

Epoch: 1 - Train Loss: 0.0485 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000
Epoch: 2 - Train Loss: 0.0355 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000
Epoch: 3 - Train Loss: 0.0346 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000
Epoch: 4 - Train Loss: 0.0340 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000
Epoch: 5 - Train Loss: 0.0336 - Learning Rate: 0.010000 - Train_IoU: 0.000 - Train_F1: 0.000


In [9]:
model.eval()
img_tns, gt = train_set[2]
print(img_tns.shape)
img_tns_ = img_tns.unsqueeze(0)
print(img_tns_.shape)
img_tns_ = img_tns_.to('cuda')
pred_mask = model(img_tns_)
print(pred_mask.shape)

torch.Size([3, 640, 640])
torch.Size([1, 3, 640, 640])
torch.Size([1, 1, 640, 640])


In [6]:
# pred_mask = torch.argmax(pred_mask, dim=1).unsqueeze(1).repeat(1,3,1,1).float()
# # print(pred_mask[0].unique())
# print(pred_mask.shape)
# print(gt["gt_tensor"])
# print(gt["gt_tensor"].unique())

In [10]:
pred_mask = torch.squeeze(pred_mask)
pred_mask = utils.toImagearr(pred_mask)
img_tns = utils.toImagearr(img_tns)
print(pred_mask.shape)
print(img_tns.shape)
utils.disp_img(image = img_tns, name = 'Original Image')
utils.disp_img(image = pred_mask, name = 'Original Image')
# img_tns.shape

(640, 640)
(640, 640, 3)
